In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from tqdm import tqdm

# Step 1: 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 2: 数据预处理
# 使用 ImageNet 数据集标准化参数
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet-50 输入大小为 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet 标准化
])

# Step 3: 加载 ImageNet 数据集
# ImageNet 官方数据集需要手动下载，或者使用 Tiny-ImageNet 作为替代
# 这里以 Tiny-ImageNet 为例（需提前下载到 ./data/tiny-imagenet-200）
dataset_dir = '/share/wangsiyuan-local/datasets/imagenet'
test_dir = f"{dataset_dir}/val5000"  # 假设 Tiny-ImageNet 验证集位于该路径

# 加载验证集
test_dataset = torchvision.datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

print(f"Test dataset size: {len(test_dataset)}")

# Step 4: 加载预训练的 ResNet-50 模型
# 加载基于 ImageNet 训练的预训练模型
model = resnet50(pretrained=True)
model = model.to(device)
model.eval()  # 设置为评估模式

# Step 5: 在测试集上评估模型
def evaluate_model(model, test_loader, device):
    model.eval()  # 确保模型在评估模式
    correct = 0
    total = 0
    with torch.no_grad():  # 禁用梯度计算
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            
            # 前向传播
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            # 计算准确率
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Accuracy on the test dataset: {accuracy:.2f}%")

# 评估模型
evaluate_model(model, test_loader, device)


Using device: cuda
Test dataset size: 50000


Evaluating: 100%|██████████| 500/500 [00:59<00:00,  8.39it/s]

Accuracy on the test dataset: 74.55%


In [ ]:
import torch
from torchvision.models import resnet50

# Step 1: 加载预训练模型
model = resnet50(pretrained=True)
model.eval()  # 设置为评估模式

# Step 2: 创建一个示例输入
# ResNet-50 的输入大小为 [batch_size, 3, 224, 224]，其中 3 是 RGB 通道数
dummy_input = torch.randn(1, 3, 224, 224)

# Step 3: 导出为 ONNX 模型
onnx_file_path = "resnet50.onnx"
torch.onnx.export(
    model,                          # PyTorch 模型
    dummy_input,                    # 示例输入
    onnx_file_path,                 # 导出的 ONNX 文件路径
    export_params=True,             # 导出训练好的参数权重
    opset_version=13,               # ONNX opset 版本
    do_constant_folding=True,       # 是否进行常量折叠优化
    input_names=["input"],          # 输入节点名称
    output_names=["output"],        # 输出节点名称
    dynamic_axes={                  # 动态轴支持（可选）
        "input": {0: "batch_size"}, # 批量大小可以动态变化
        "output": {0: "batch_size"}
    }
)

print(f"Model exported to {onnx_file_path}")
